In [ ]:
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as Expected
from selenium.webdriver.common.alert import Alert

# 消す範囲設定
START_PAGE = 2
END_PAGE = 100
# START_PAGE <= page < END_PAGE に記載された日記を削除する。
# 各ページには25の日記が含まれ、Page=1が最新となる。
# START_PAGE= 2を指定することで、1p目（最新の25件）をターゲットから外せる。
# ENDを大きくし過ぎると遅くなるため、小刻みに消したほうがいい。


# ログイン情報…ここを自分の情報に書き換える
USER_ID = 'your.name@please.xxx'
PASSWORD = 'password'


# 初期化からログインまで
# Chromeを起動する
driver = selenium.webdriver.Chrome()

# ログインページを開く
driver.get('https://hatelabo.jp/login?backurl=https%3A%2F%2Fanond%2Ehatelabo%2Ejp%2F')

#名前とIDを入力する
q = driver.find_element(By.NAME, 'key')
q.send_keys(USER_ID)

q = driver.find_element(By.NAME, 'password')
q.send_keys (PASSWORD)

q.submit()

WebDriverWait(driver, timeout=10).until(Expected.title_contains('の日記'))    

user_name = driver.find_element(By.XPATH, '//td[@class="username"]/a').text
print ("User:" + user_name)



In [ ]:

# エントリリストの取得

edit_url_list = []

for page in range(START_PAGE, END_PAGE):

    driver.get('https://anond.hatelabo.jp/' + user_name + '/?page=' + str(page))

    article_list = driver.find_elements(By.CLASS_NAME, 'section')

    if (len(article_list) <= 0):
        break

    for article in article_list:
        
        # Title
        #print (article.find_element(By.XPATH, 'h3/a[2]').text)
        
        # 言及数表示：削除条件に入れる？（条件にする場合は正規表現で数字を取り出す）
        #print (article.find_element(By.XPATH, 'p[@class="sectionfooter"]/a[2]').text)

        #ブクマ数取得方法は難しすぎてあきらめた

        x = article.find_element(By.CLASS_NAME, 'edit').get_attribute('href')
        edit_url_list.append(x)
        # print(x)

# 最終ページ
print ('Page: ' + str(page-1))

# page1で終了：＝エントリが無い
if (page <= 1):
    print('no articles.')
    sys.exit()
    
print ('Article: ' + str(len(edit_url_list)))



In [ ]:
# 削除を実行する…編集用画面を開いて削除ボタンを押す
for edit_url in edit_url_list:

    driver.get(edit_url)

    q = driver.find_element(By.CLASS_NAME, 'delete-button')
    #q.click
    q.click()

    WebDriverWait(driver, timeout=10).until(Expected.presence_of_all_elements_located)
    Alert(driver).accept()

    # ページ読み込みでウェイトを入れる
    driver.get('https://anond.hatelabo.jp/' + user_name + '/?page=1')
    WebDriverWait(driver, timeout=10).until(Expected.presence_of_all_elements_located)
    

print('done')
